In [92]:
# archivo:    spsi.p6.rsa
# asignatura: Seguridad y Protección de Sistemas Informáticos
# práctica:   Práctica 6
# autor:      José María Martín Luque

In [93]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

# Ejercicio 1

*Emita una pareja de claves, pública y privada, para adherirse usted a un círculo RSA como usuario $A$.*

Necesitamos:

- Dos números $p$, $q$ primos.
- El producto $n = pq$.
- El valor $\phi(n)$.
- Un valor $e$ tal que $(e, \phi(n)) = 1$.
- Un valor $d$ tal que $ed \equiv 1 \mod \phi(n)$, es decir, $d = e^{-1}\mod\phi(n)$.

La clave pública será la tupla $(n, e)$ y la clave privada, $(p, q, d)$.

In [94]:
p_A = (2^19) - 1
print(p_A)
print(is_prime(p_A))

524287
True


In [95]:
q_A = (2^31) - 1
print(q_A)
print(is_prime(q_A))

2147483647
True


In [96]:
n_A = p_A*q_A; n_A

1125897758834689

Calculamos $\phi(n)$ sabiendo que $\phi(n) = (p-1)(q-1)$.

In [97]:
phi_A = (p_A - 1)*(q_A - 1); phi_A

1125895610826756

Para buscar $e$ generamos enteros aleatorios menores que $\phi(n)$ con la función `random_element` hasta encontrar alguno tal que $(e, \phi(n)) = 1$.

In [98]:
def genera_e(phi):
    e = ZZ.random_element(phi)
    while gcd(e, phi) != 1:
        e = ZZ.random_element(phi)
    return e

In [99]:
e_A = genera_e(phi_A); e_A

294928947328891

In [100]:
gcd(e_A, phi_A)

1

In [101]:
def genera_d(e, phi):
    bezout = xgcd(e, phi); bezout
    d = Integer(mod(bezout[1], phi))
    return d

In [102]:
d_A = genera_d(e_A, phi_A) ; d_A

338903956284055

In [103]:
mod(d_A * e_A, phi_A)

1

La clave pública de $A$ va a ser la tupla $(n, e)$.

In [104]:
pub_A = (n_A, e_A); pub_A

(1125897758834689, 294928947328891)

La clave privada de $A$ va a ser la tupla $(p, q, d)$.

In [105]:
priv_A = (p_A, q_A, d_A); priv_A

(524287, 2147483647, 338903956284055)

# Ejercicio 2

*Simule ser un segundo usuario $B$, del mismo círculo y envíe de $A$ a $B$ un mensaje firmado y cifrado. Al recibirlo descífrelo y compruebe la identidad del emisor.*

A continuación se describen algunas funciones auxiliares necesarias para codificar un mensaje.

In [106]:
def completa(texto, m):
    """Completa el texto para que su longitud sea múltiplo de 'm'"""
    resto = len(texto) % m
    r = ""
    
    if resto:
        resto = m - resto
    
    for i in range(resto):
        r = r + "w"
        
    return texto + r

In [107]:
def texto_a_numeros(texto):
    """Convierte el texto recibido a una cadena de números.
    
    Cada número que representa la posición de cada carácter en el alfabeto —sin 
    la ñ ni acentos—, que es reemplazada por ny.

    El texto se codifica en minúscula.
    """

    texto = texto.lower()
    texto = texto.replace("ñ", "ny")
    texto = texto.replace("á", "a")
    texto = texto.replace("é", "e")
    texto = texto.replace("í", "i")
    texto = texto.replace("ó", "o")
    texto = texto.replace("ú", "u")

    numeros = []
    for c in texto:
        n = ord(c) - 97
        numeros.append(n)
    return numeros

In [108]:
def numeros_a_texto(numeros):
    """Convierte la lista de números recibida en un string entendiendo que cada 
    número es la posición de una letra en el alfabeto.
    """

    texto = ""
    for n in numeros:
        c = chr(n + 97)
        texto = texto + c
    return texto

In [109]:
def bloques(lista, m):
    """Devuelve bloques de 'm' elementos a partir de 'lista'."""
    for i in range(0, len(lista), m):
        yield lista[i:i + m]

Tenemos que generar las claves de $B$.

In [110]:
p_B = (2^61) - 1
print(p_B)
print(is_prime(p_B))

2305843009213693951
True


In [111]:
q_B = (2^89) - 1
print(q_B)
print(is_prime(q_B))

618970019642690137449562111
True


In [117]:
n_B = p_B*q_B
phi_B = (p_B - 1)*(q_B - 1)
e_B = genera_e(phi_B)
d_B = genera_d(e_B, phi_B) ; d_B

1418610524977236755080838127275432279717913493

In [118]:
pub_B = (n_B, e_B); pub_B

(1427247692705959880439315947500961989719490561,
 425004578570056476170555047107321620563474357)

In [119]:
priv_B = (p_B, q_B, d_B); priv_B

(2305843009213693951,
 618970019642690137449562111,
 1418610524977236755080838127275432279717913493)